**Name: LAKSHMI SRI RUPA KURUKURI**

**Group: 09**

Importing libraries

In [257]:
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
import random
import warnings
from sklearn.pipeline import make_pipeline
from scipy.linalg.blas import ddot, daxpy, dscal

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC

`Exercise Question`

In [258]:
X1 = [{'city':'Gothenburg', 'month':'July'},
      {'city':'Gothenburg', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y1 = ['rain', 'rain', 'sun', 'rain']

X2 = [{'city':'Sydney', 'month':'July'},
      {'city':'Sydney', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y2 = ['rain', 'sun', 'sun', 'rain']

classifier1 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
classifier1.fit(X1, Y1)
guesses1 = classifier1.predict(X1)
print(accuracy_score(Y1, guesses1))

classifier2 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
#classifier2 = make_pipeline(DictVectorizer(), LinearSVC())
classifier2.fit(X2, Y2)
guesses2 = classifier2.predict(X2)
print(accuracy_score(Y2, guesses2))

1.0
0.5


1. `Why could the classifier "memorize" the training data in the first case, but not in the second case?`

    In the first training set, the city Gothenburg always has the result as rain which is clear and simple. This simple relationship between the data always allows the perceptron to easily memorize the training data. In the second training set, the weather depends on both the city and monht, which is a complex type of relationship and is not linear, which makes the algorithm to not memorize the relations easily.

2. `Why can't we improve the classifier performance by switching to a LinearSVC?`

    Switching to a LinearSVC model does not imporve the accuracy because just like perceptron, LinearSVC is also a linear Classifier. As mentioned in the previous answer, the issue with the second data set is the non-linear relationship between variables or features and the output. Therefore, as both the models are constrained to linearity, they cannot accurately catch non-linear patterns.



1.   A new class named "PerceptronTransformer()" is created transform output of the perceptron class to a binary matrix.
2.   Another class named pegasos_calssifier() is created using the pseudocode given in the pegasos resaerch work.



In [259]:
"""This file shows a couple of implementations of the perceptron learning
algorithm. It is based on the code from Lecture 3, but using the slightly
more compact perceptron formulation that we saw in Lecture 6.

There are two versions: Perceptron, which uses normal NumPy vectors and
matrices, and SparsePerceptron, which uses sparse vectors and matrices.
The latter may be faster when we have high-dimensional feature representations
with a lot of zeros, such as when we are using a "bag of words" representation
of documents.
"""

warnings.filterwarnings("ignore")

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x

class PerceptronTransformer(LinearClassifier):
    """
    A custom class to transform output of the perceptron class to a binary matrix.
"""
    def __init__(self, perceptron):
        self.perceptron = perceptron

    def fit(self, X, y):
        self.perceptron.fit(X, y)
        return self

    def transform(self, X):
        # Predict class labels
        predictions = self.perceptron.predict(X)
        predictions_numerical = np.where(predictions == 'positive', 1, -1)
        # Return predictions as binary matrix (1 for positive, 0 for negative)
        return np.array(predictions_numerical.astype(int))

class pegasos_classifier:
  """
  Pegasos algorithm for training a Support Vector Classifier (SVC).
  """
  def __init__(self, eta=0.01, lambdaa=0.01, iterations=100):
    self.eta = eta  # Learning rate
    self.lambdaa = lambdaa  # Regularization parameter
    self.iterations = iterations  # Maximum number of iterations
    self.w = None  # Weight vector


  # Function to use the fit() method for the pegosos_classifier
  def fit(self, X, y):
    """
    Trains the Pegasos SVC model on the given data (X, y).

    Args:
      X: A numpy array of shape (n_samples, n_features) representing the training data.
      y: A numpy array of shape (n_samples,) representing the labels (either +1 or -1).
    """
    X = np.array(X)
    if len(X.shape) < 2:
            X = np.atleast_2d(X).T
    n_samples, features = X.shape
    self.w = np.zeros(features)

    for _ in range(self.iterations):
      for i in range(n_samples):
        Xi = X[i]
        Yi = y[i]

        Xi = Xi.astype(np.float64)
        if Yi * np.dot(Xi, self.w) < 1:

            self.w = (1 - self.eta * self.lambdaa) * self.w + self.eta * Yi * Xi
        else:
            self.w = (1 - self.eta * self.lambdaa) * self.w


  # Function to use the predict() method for the pegosos_classifier
  def predict(self, X):
    """
    Predicts the labels for the given data (X).

    Args:
      X: A numpy array of shape (n_samples, n_features) representing the data to predict.

    Returns:
      A numpy array of shape (n_samples,) containing the predicted labels (+1 or -1).
    """
    if len(X.shape)<2:
        X = np.atleast_2d(X).T
    return np.sign(np.dot(X, self.w))


  # Function for hpper-parameter tuning.
  def set_params(self, **params):
    """
    Set hyperparameters of the PegasosSVC model.
    """
    for key, value in params.items():
      setattr(self, key, value)
    return self

In [260]:
# Function to read data (remains unchanged)
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y

X,Y = read_data('data/all_sentiment_shuffled.txt')

Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.1,random_state=0)

Implementing `preceptron`. Note that this is the perceptron model given in the `aml_perceptron`, not the `sklearn's perceptron`.

In [261]:
piplineperceptron = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=10000),
    Normalizer(),
    Perceptron()
)

t0 = time.time()
piplineperceptron.fit(Xtrain,Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

Yguess = piplineperceptron.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)));

Training time: 5.96 sec.
Accuracy: 0.7953.


Implementation of Pegasos algorithm with a `SVC` model.

In [262]:
class PegasosSVC(LinearClassifier):

    def __init__(self, n_iter = 20, regularisation = 1):
        if not isinstance(n_iter, int):
            raise TypeError('n_iter must be an integer.')
        self.n_iter = n_iter
        self.regularisation = regularisation
    
    def fit(self, x,y):
        self.find_classes(y)
        Ye = self.encode_outputs(y)
        if not isinstance(x, np.ndarray):
            x = x.toarray()
        
        size_x, n_features = x.shape[0],x.shape[1]
        self.w = np.zeros(n_features)

        for i in range(self.n_iter):
            r = np.random.choice(size_x,1)[0]

            learning_rate = 1.0/ (self.regularisation*(i+1))

            X,Y = x[r], Ye[r]
            score = self.w.dot(X)

            if Y*score<1:
                self.w = (1 - learning_rate*self.regularisation)*self.w + learning_rate*Y*X
            else:
                self.w = (1 - learning_rate*self.regularisation)*self.w

piplinePegososSVC = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    PegasosSVC(n_iter=10000, regularisation=1.0/len(Xtrain)*10)
)

t0 = time.time()
piplinePegososSVC.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = piplinePegososSVC.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.53 sec.
Accuracy: 0.7978.


Implementation of Pegasos algorithm with a `Logistic regression` model.

In [263]:
class PegasosLR(LinearClassifier):

    def __init__(self, n_iter=20,reg_lambda=1):
        if not isinstance(n_iter, int):
            raise TypeError('n_iter must be an integer.')
        self.n_iter = n_iter
        self.reg_lambda = reg_lambda
        # self.regularisation = regularisation
    
    def fit(self,X,Y):
        self.find_classes(Y)
        Ye = self.encode_outputs(Y)
        if not isinstance(X, np.ndarray):
            X = X.toarray()
        
        size_X, n_features = X.shape[0], X.shape[1]
        self.w = np.zeros(n_features)

        for i in range(self.n_iter):
            k = np.random.choice(size_X,1)[0]
            learning_rate = 1.0 / (self.reg_lambda*(i+1))

            x,y = X[k], Ye[k]

            score = x.dot(self.w)

            self.w = (1 - learning_rate*self.reg_lambda)*self.w + x*learning_rate*(y/(1+1/(1+np.exp(-y*score))))

piplinePegososLR = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    PegasosLR(n_iter=10000, reg_lambda=1.0/len(Xtrain)*10)
)

t0 = time.time()
piplinePegososLR.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = piplinePegososLR.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Training time: 1.39 sec.
Accuracy: 0.7643.


Let us implement `BLAS` as mentioned in the optional tasks to decrease the time and improve the accuracy. 

In [264]:
class PegasosSVC_optional(LinearClassifier):
    def __init__(self, n_iter = 20, reg_lambda=1):
        if not isinstance(n_iter, int):
            raise TypeError('n_iter should be an integer.')

        self.n_iter = n_iter
        self.reg_lambda = reg_lambda

    def fit(self,X,Y):
        self.find_classes(Y)
        Ye = self.encode_outputs(Y)
        if not isinstance(X, np.ndarray):
            X = X.toarray() 

        size_x , n_features = X.shape[0], X.shape[1]
        self.w = np.zeros(n_features)


        for i in range(self.n_iter):

            k = np.random.choice(size_x,1)[0]

            eta = 1.0 / (self.reg_lambda*(i+1))

            x,y = X[k], Ye[k]

            score = ddot(self.w,x)

            if y*score < 1:
                dscal(1 - eta*self.reg_lambda,self.w)
                daxpy(y*x, self.w, a = eta)
            else:
                dscal(1 - eta*self.reg_lambda,self.w)


pipelinePegasosSVC_optional = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    PegasosSVC_optional(n_iter=10000, reg_lambda=1.0/len(Xtrain)*10)
)

t0 = time.time()
pipelinePegasosSVC_optional.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

Yguess = pipelinePegasosSVC_optional.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 1.55 sec.
Accuracy: 0.7995.


Finally, after implementing `BLAS` on the `PegasosSVC`, we have decreased the time by around `250ms`. Using NumPy arrays might be the reason for optimization.